## 1. Importação das bibliotecas

In [1]:
import heapq
import json
import math
import nltk
import os
import pandas as pd
import spacy
import string
import time

## 2. Pré-Processamento

### 2.1. Definição da função de Pré-processamento

In [2]:
def remove_non_ascii(texto):
    return "".join(c for c in texto if ord(c) < 128)

In [3]:
def remove_special_characters(texto):
    return "".join(c for c in texto if c.isalnum() or c.isspace())

In [4]:
def remove_digits(texto):
    return "".join(c for c in texto if not c.isdigit())

A função **preprocessamento** faz a limpeza no texto removendo todos os caracteres que devem ser desconsiderados no processo de geração de tokens de sentenças e palavras.

In [5]:
#!python -m spacy download en

In [6]:
def preprocessamento(texto):
    # Lista de palavras irrelevantes para a etapa de processamento do texto.
    stopwords = nltk.corpus.stopwords.words('english')

    # Lista de sinais de pontuação
    punctuation = string.punctuation

    #pln = spacy.load('en_core_web_sm')

    texto = texto.lower()
    texto = remove_non_ascii(texto)
    texto = remove_special_characters(texto)
    texto = remove_digits(texto)
    
    #doc = pln(texto)
    
    tokens = []
    # Pré-processamento sem a lematização
    for token in nltk.word_tokenize(texto):
        tokens.append(token)
    
    # Pré-processamento com a lematização
    #for token in documento:
        #tokens.append(token.lemma_)

    # Filtrar apenas as palavras que não estiverem na lista de stopwords e nem na lista de pontuações
    tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in punctuation]
    
    # Converter a lista de palavras para uma String, não adiciona valores numéricos
    texto = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

    return texto

### 2.2. Definição das funções de tokenização

A função **token_sentences** transforma cada sentença do texto original em um token. Cada frase do texto, é considerado um token.

In [7]:
# Transforma cada frase do texto original em um token.
def token_sentences(texto):
    sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
    
    return sentencas_originais

A função **preprocessing_sentences** recebe uma lista de sentenças com o texto original e executa a função de preprocessamento, eliminando caracteres especiais, sinais de pontuação, dígitos, stop words, etc. Essa função retorna uma lista de sentenças formatadas (pré-processadas)

In [8]:
# Pré-processamento das sentenças originais
def preprocessing_sentences(sentencas_originais):
    sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]
    
    return sentencas_formatadas

## 3. Algoritmo Baseado em Frequência

### 3.1. Definição da função summarize

A função **summarize** executa os seguintes passos:
- Calcula a frequência das palavras no texto
- Calcula a nota de cada sentença baseada na frequência de palavras.
- Gera uma lista com as melhores sentenças descritas no texto original, ou seja, sem as exclusões realizadas na etapa do pré-processamento.

Os seguintes parâmetros são passados para a função:
- texto_original: conteúdo com o texto original, sem pré-processamento;
- texto_formatado: texto original pré-processado;
- quantidade_setencas: Número de sentenças que irão compor o resumo do texto.

Observação: Este algoritmo considera que a quantidade de sentenças que irão compor o resumo corresponde a 40% do número total de sentenças do texto. (Esse valor pode ser alterado na etapa dos testes)

In [9]:
def summarize(texto_original, texto_formatado, quantidade_sentencas):
    
    # Cálculo das frequências das palavras
    frequencia_palavras = nltk.FreqDist(nltk.word_tokenize(texto_formatado))
    frequencia_maxima = max(frequencia_palavras.values())
    
    for palavra in frequencia_palavras.keys():
        # Frequência proporcional das palavras
        frequencia_palavras[palavra] = (frequencia_palavras[palavra] / frequencia_maxima)
    lista_sentencas = nltk.sent_tokenize(texto_original)
    
    # Cálculo das notas das sentenças
    nota_sentencas = {}
    for sentenca in lista_sentencas:
        for palavra in nltk.word_tokenize(sentenca.lower()):
            if palavra in frequencia_palavras.keys():
                if sentenca not in nota_sentencas.keys():
                    nota_sentencas[sentenca] = frequencia_palavras[palavra]
                else:
                    nota_sentencas[sentenca] += frequencia_palavras[palavra]
    
    # Cálculo das melhores sentenças
    melhores_sentencas = heapq.nlargest(quantidade_sentencas, nota_sentencas, key=nota_sentencas.get)
    
    return lista_sentencas, melhores_sentencas, nota_sentencas

### 3.2. Definição da função view_summary

A função **view_summary** exibe o sumário do texto resultante após a aplicação do algoritmo de sumarização. Esta função retorna as melhores sentenças com o texto original e concatenadas. 

In [10]:
def view_summary(lista_sentencas, melhores_sentencas):
    texto = ''
    
    for sentenca in lista_sentencas:
        if sentenca in melhores_sentencas:
            texto = texto + ' ' + sentenca
    return texto

### 3.3. Execução do Algoritmo para todas as notícias

Esse trecho de código corresponde à execução completa do **Algoritmo de Frequência** para um lote de notícias. Os seguintes passos são executados:



In [11]:
# Leitura do arquivo JSON com a base de dados de notícias
dt = pd.read_json("news.json")
display(dt)

,titulo,conteudo,sumario
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...
1,Malaika slams user who trolled her for 'divorc...,"From her special numbers to TVappearances, Bol...",Malaika Arora slammed an Instagram user who tr...
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...
...,...,...,...
3960,Rasna seeking ?250 cr revenue from snack categ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",Fruit juice concentrate maker Rasna is eyeing ...
3961,Sachin attends Rajya Sabha after questions on ...,Former cricketer Sachin Tendulkar was spotted ...,Former Indian cricketer Sachin Tendulkar atten...
3962,Shouldn't rob their childhood: Aamir on kids r...,"Aamir Khan, whose last film Dangal told the st...","Aamir Khan, while talking about reality shows ..."
3963,"Asha Bhosle gets ?53,000 power bill for unused...",Maharahstra Power Minister Chandrashekhar Bawa...,The Maharashtra government has initiated an in...


In [12]:
# Lista para armazenar o resultado do processamento do Algoritmo de frequência para todas as notícias
algoritmo_frequencia = []
processamento_frequencia = []

inicio = time.time()
processamento_frequencia.append({'Begin': inicio})
print('Begin:', inicio)

for i in range(len(dt)):
    titulo = dt.iloc[i, 0]
    conteudo = dt.iloc[i, 1]
    sumario = dt.iloc[i, 2]
    
    # Geração de tokens (sentenças) com os textos originais
    sentencas_originais = token_sentences(conteudo)
    
    # Executar o pre-processamento das sentenças para eliminar os caracteres e palavras irrelevantes
    texto_formatado = preprocessamento(conteudo)
    
    # Quantidade de sentenças que irão compor o sumário: 20% do total de sentenças
    quantidade_sentencas = math.ceil(len(sentencas_originais)*0.20)
    
    # Geração das melhores sentenças que irão compor o sumário do texto
    lista_sentencas, melhores_sentencas, nota_sentencas = summarize(conteudo, texto_formatado, quantidade_sentencas)
    
    # Geração do sumário somente com as frases correspondentes às melhores sentenças do texto
    sumario_frequencia = view_summary(sentencas_originais, melhores_sentencas)
    
    algoritmo_frequencia.append({'index': i, 'titulo': titulo, 'conteudo': conteudo, 'sumario': sumario, 'sumario_frequencia': sumario_frequencia})
    if i % 100 == 0:
        print(i)

fim = time.time()
print('Fim da execução:', fim)
processamento_frequencia.append({'End': fim})

duracao = fim - inicio
print('Duração:', duracao)
processamento_frequencia.append({'Duration': duracao})

Begin: 1635870196.4387906
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
Fim da execução: 1635870228.2936702
Duração: 31.85487961769104


In [13]:
# Será criado um arquivo algoritmo_frequencia.json no mesmo diretório de execução do programa
arquivo_gravar = os.path.join('algoritmo_frequencia.json')
arquivo = open(arquivo_gravar, 'w+')
arquivo.write(json.dumps(algoritmo_frequencia, indent=1))
arquivo.close()

In [14]:
# Será criado um arquivo processamento_frequencia.json no mesmo diretório de execução do programa
arquivo_gravar = os.path.join('processamento_frequencia.json')
arquivo = open(arquivo_gravar, 'w+')
arquivo.write(json.dumps(processamento_frequencia, indent=1))
arquivo.close()